In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install torchsummary

In [1]:
from torch import nn
from torch.utils.data import dataloader
import torch
import torchvision
import torchvision.transforms as transforms
import math
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [2]:
class block(nn.Module):
    def calc_same_pad(self, i: int, k: int, s: int, d: int) -> int:
        return max((math.ceil(i / s) - 1) * s + (k - 1) * d + 1 - i, 0)
    
    def __init__(self,in_channels,out_channels,stride=1,downsampling = None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=3,padding="same",stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=stride, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.downsamp = downsampling
        self.stride = stride
        self.topad = True
        if self.downsamp is not None:
            self.topad = False
            self.stride = 1
        
    def forward(self,x):
        identity = x
        out = x
        # print(out.size())
        out = self.conv1(out)
        out = self.bn1(out)
        out = self.relu(out)
        # print(out.size())
        ih, iw = x.size()[-2:]
        pad_h = self.calc_same_pad(i=ih, k=3, s=self.stride, d=1)
        pad_w = self.calc_same_pad(i=iw, k=3, s=self.stride, d=1)
        if (pad_h > 0 or pad_w > 0):
            out= nn.functional.pad(
                out, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2]
            )
        # print("out size : ",out.size())
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsamp is not None:
            identity = self.downsamp(x)
        # print(out.size())
        # print("out size after : ",out.size())
        # print("identity size :",identity.size())
        out += identity
        out = self.relu(out)
        return out

In [3]:
class resnet(nn.Module):
    def __init__(self,n,r):
        super().__init__()
        cluster1 = []
        for i in range(n-1):
            cluster1.append(block(16,16))
        down1 = self.downsample = nn.Sequential(
                nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
                nn.BatchNorm2d(32)
                )
        cluster1.append(block(16,32,stride = 2,downsampling = down1))
        self.c1 = nn.Sequential(*cluster1)

        cluster2 = []
        for i in range(n-1):
            cluster2.append(block(32,32))
        down2 = self.downsample = nn.Sequential(
                nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
                nn.BatchNorm2d(64)
                )
        cluster2.append(block(32,64,stride = 2,downsampling = down2))
        self.c2 = nn.Sequential(*cluster2)

        cluster3 = []
        for i in range(n):
            cluster3.append(block(64,64))
        self.c3 = nn.Sequential(*cluster3)
        self.conv0 = nn.Conv2d(3,16,kernel_size=3,stride=1,padding="same", bias=False)
        self.bn0 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.avg_pool = nn.AvgPool2d(kernel_size=2,stride=2)
        self.output = nn.Linear(64*32*32,r)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self,x):
        out = self.conv0(x)
        # print(out.size())
        out = self.bn0(out)
        # print(out.size())
        out = self.relu(out)
        # print(out.size())
        out = self.c1(out)
        out = self.c2(out)
        out = self.c3(out)
        out = self.avg_pool(out)
        out = out.reshape(-1,64*32*32)
        # print(out.shape)
        out = self.output(out)
        # out = self.softmax(out)
        # print(out)
        return out

In [4]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Resize(256)]
#      )

# batch_size = 32

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
import torch.optim as optim
net = resnet(2,25)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
from torchsummary import summary
summary(net,(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 256, 256]             432
       BatchNorm2d-2         [-1, 16, 256, 256]              32
              ReLU-3         [-1, 16, 256, 256]               0
            Conv2d-4         [-1, 16, 256, 256]           2,304
       BatchNorm2d-5         [-1, 16, 256, 256]              32
              ReLU-6         [-1, 16, 256, 256]               0
            Conv2d-7         [-1, 16, 256, 256]           2,304
       BatchNorm2d-8         [-1, 16, 256, 256]              32
              ReLU-9         [-1, 16, 256, 256]               0
            block-10         [-1, 16, 256, 256]               0
           Conv2d-11         [-1, 32, 256, 256]           4,608
      BatchNorm2d-12         [-1, 32, 256, 256]              64
             ReLU-13         [-1, 32, 256, 256]               0
           Conv2d-14         [-1, 32, 1

In [6]:
# for epoch in range(2):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         data
#         inputs, labels = data
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         # print(labels)
#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs)
#         # outputs = torch.argmax(outputs,dim=1)
#         # for i in range(outputs.shape[0]):
#         #     outputs[i] = torch.argmax(outputs[i])
#         # print(outputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 10 == 9:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
#             running_loss = 0.0

# print('Finished Training')

In [7]:
# correct = 0
# total = 0
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         images = images.to(device)
#         labels = labels.to(device)
#         # calculate outputs by running images through the network
#         outputs = net(images)
#         # the class with the highest energy is what we choose as prediction
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [8]:
classes = ['Asian-Green-Bee-Eater','Brown-Headed-Barbet','Cattle-Egret','Common-Kingfisher', 'Common-Myna', 'Common-Rosefinch', 'Common-Tailorbird', 'Coppersmith-Barbet', 'Forest-Wagtail', 'Gray-Wagtail', 'Hoopoe', 'House-Crow', 'Indian-Grey-Hornbill', 'Indian-Peacock', 'Indian-Pitta', 'Indian-Roller', 'Jungle-Babbler', 'Northern-Lapwing', 'Red-Wattled-Lapwing', 'Ruddy-Shelduck', 'Rufous-Treepie', 'Sarus-Crane', 'White-Breasted-Kingfisher', 'White-Breasted-Waterhen', 'White-Wagtail']
classes = sorted(classes)
nametonum = {}
numtoname = {}
for i in range(25):
    nametonum[classes[i]] = i
    numtoname[i] = classes[i]
print(classes)

['Asian-Green-Bee-Eater', 'Brown-Headed-Barbet', 'Cattle-Egret', 'Common-Kingfisher', 'Common-Myna', 'Common-Rosefinch', 'Common-Tailorbird', 'Coppersmith-Barbet', 'Forest-Wagtail', 'Gray-Wagtail', 'Hoopoe', 'House-Crow', 'Indian-Grey-Hornbill', 'Indian-Peacock', 'Indian-Pitta', 'Indian-Roller', 'Jungle-Babbler', 'Northern-Lapwing', 'Red-Wattled-Lapwing', 'Ruddy-Shelduck', 'Rufous-Treepie', 'Sarus-Crane', 'White-Breasted-Kingfisher', 'White-Breasted-Waterhen', 'White-Wagtail']


In [ ]:
train_dataset = []
train_targets = []

import cv2
import os
for i in range(25):
    folder = "E:/col775/Birds_25/train/"+classes[i]
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            train_dataset.append(img)
            train_targets.append(i)
print("done")
train_dataset = torch.Tensor(train_dataset).to(device)
train_targets = torch.Tensor(train_targets).to(device)

In [ ]:
val_dataset = []
val_targets = []

import cv2
import os
for i in range(25):
    folder = "E:/col775/Birds_25/val/"+classes[i]
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            val_dataset.append(img)
            val_targets.append(i)
val_dataset = torch.Tensor(val_dataset)
val_targets = torch.Tensor(val_targets)

In [ ]:
test_dataset = []
test_targets = []

import cv2
import os
for i in range(25):
    folder = "E:/col775/Birds_25/test/"+classes[i]
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            test_dataset.append(img)
            test_targets.append(i)
test_dataset = torch.Tensor(test_dataset)
test_targets = torch.Tensor(test_targets)

In [14]:
# import torch
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# import numpy as np
# from PIL import Image


# class MyDataset(Dataset):
#     def __init__(self, data, targets, transform=None):
#         self.data = data
#         self.targets = torch.LongTensor(targets)
#         self.transform = transform
        
#     def __getitem__(self, index):
#         x = self.data[index]
#         y = self.targets[index]
        
#         if self.transform:
#             x = Image.fromarray(self.data[index].astype(np.uint8).transpose(1,2,0))
#             x = self.transform(x)
        
#         return x, y
    
#     def __len__(self):
#         return len(self.data)

In [16]:
# import random
# def shuffler (arr, n):
     
#     # We will Start from the last element 
#     # and swap one by one.
#     for i in range(n-1,0,-1):
         
#         # Pick a random index from 0 to i
#         j = random.randint(0,i+1)
         
#         # Swap arr[i] with the element at random index
#         arr[i],arr[j] = arr[j],arr[i]
#     return arr
# def make_mini_batch(data,targets,batch_size):
#     # print(data.shape)
#     num_batches = int(len(targets)/batch_size)
#     aug = []
#     new_data = [[] for i in range(len(targets))]
#     new_targets = [-1 for i in range(len(targets))]
#     for i in range(len(targets)):
#         aug.append(i)
#     aug = shuffler(aug,len(targets))
#     print(aug[0])
#     # for i in range(len(targets)):
#     #     new_data[i] = aug[i][0]
#     #     new_targets[i] = aug[i][1]
#     batch_data = [[] for i in range(num_batches)]
#     batch_targets = [[] for i in range(num_batches)]
#     for i in range(len(targets)):
#         batch_data[int(i/len(targets))].append(data[aug[i]])
#         batch_targets[int(i/len(targets))].append(targets[aug[i]])
#     batch= [[] for i in range(num_batches)]
#     for i in range(num_batches):
#         batch[int(i)].append([torch.Tensor(batch_data[i]),torch.Tensor(batch_targets[i])])
#     return batch

In [ ]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Resize(256)]
#      )

# batch_size = 8
# train_set = MyDataset(train_dataset,train_targets,transforms)
# val_set = MyDataset(val_dataset,val_targets,transforms)
# test_set = MyDataset(test_dataset,test_targets,transforms)
# # trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True, num_workers=8)
# trainloader = make_mini_batch(train_dataset,train_targets,32)
# # valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,shuffle=True, num_workers=2)
# # testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=True, num_workers=2)


In [ ]:
losses = {}
prev_val_loss = 1e10

In [ ]:
for data in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0], data[1]
        inputs = inputs.to(device)
        labels = labels.to(device)
        print(labels)

In [ ]:
# n_epochs = 100 # or whatever
# batch_size = 128 # or whatever

# for epoch in range(n_epochs):

#     # X is a torch Variable
#     permutation = torch.randperm(X.size()[0])

#     for i in range(0,X.size()[0], batch_size):
#         optimizer.zero_grad()

#         indices = permutation[i:i+batch_size]
#         batch_x, batch_y = X[indices], Y[indices]

#         # in case you wanted a semi-full example
#         outputs = model(batch_x)
#         loss = lossfunction(outputs,batch_y)

#         loss.backward()
#         optimizer.step()


In [ ]:
n_epochs = 100 # or whatever
batch_size = 32 # or whatever
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    permutation = torch.randperm(train_targets.size()[0])
    for i in range(0,X.size()[0], batch_size):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = train_data[i:i+batch_size],train_targets[i:i+batch_size]
        inputs = inputs.to(device)
        labels = labels.to(device)
        print(labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    running_loss/=len(trainloder)
    val_loss = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(valloader)
    losses[epoch] = [running_loss,val_loss]
    if(val_loss < prev_val_loss):
        prev_val_loss = val_loss
        torch.save(net.state_dict(), r"E:\col775\model_saves")
print('Finished Training')